In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model 
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

plt.style.use('ggplot')

%matplotlib inline

# 1. Dataset

In [2]:
os.getcwd()

'D:\\Softwares\\Deep_Learning\\Tasks\\Attribute Prediction\\face-clustering\\notebook'

In [3]:
os.chdir('..')
os.getcwd()

'D:\\Softwares\\Deep_Learning\\Tasks\\Attribute Prediction\\face-clustering'

In [4]:
class CelebA():
    def __init__(self, main_folder='celeba-dataset/', selected_features=None, drop_features=[]):
        self.main_folder = main_folder
        self.images_folder   = os.path.join(main_folder, 'img_align_celeba/img_align_celeba/')
        self.attributes_path = os.path.join(main_folder, 'list_attr_celeba.csv')
        self.partition_path  = os.path.join(main_folder, 'list_eval_partition.csv')
        self.selected_features = selected_features
        self.features_name = []
        self.__prepare(drop_features)
        
    def __prepare(self, drop_features):
        '''do some preprocessing before using the data: e.g. feature selection'''
        # attributes:
        if self.selected_features is None:
            self.attributes = pd.read_csv(self.attributes_path)
            self.num_features = 40
        else:
            self.num_features = len(self.selected_features)
            self.selected_features = self.selected_features.copy()
            self.selected_features.append('image_id')
            self.attributes = pd.read_csv(self.attributes_path)[self.selected_features]
            
        # remove unwanted features:
        for feature in drop_features:
            if feature in self.attributes:
                self.attributes = self.attributes.drop(feature, axis=1)
                self.num_features -= 1
        
        self.attributes.set_index('image_id', inplace=True)
        self.attributes.replace(to_replace=-1, value=0, inplace=True)
        self.attributes['image_id'] = list(self.attributes.index)

        self.features_name = list(self.attributes.columns)[:-1]
        
        # load ideal partitioning:
        self.partition = pd.read_csv(self.partition_path)
        self.partition.set_index('image_id', inplace=True)
    
    def split(self, name='training', drop_zero=False):
        '''Returns the ['training', 'validation', 'test'] split of the dataset'''
        # select partition split:
        if name is 'training':
            to_drop = self.partition.where(lambda x: x != 0).dropna()
        elif name is 'validation':
            to_drop = self.partition.where(lambda x: x != 1).dropna()
        elif name is 'test':  # test
            to_drop = self.partition.where(lambda x: x != 2).dropna()
        else:
            raise ValueError('CelebA.split() => `name` must be one of [training, validation, test]')
        
        partition = self.partition.drop(index=to_drop.index)

        # join attributes with selected partition:
        joint = partition.join(self.attributes, how='inner').drop('partition', axis=1)

        if drop_zero is True:
            # select rows with all zeros values
            return joint.loc[(joint[self.features_name] == 1).any(axis=1)]
        elif 0 <= drop_zero <= 1:
            zero = joint.loc[(joint[self.features_name] == 0).all(axis=1)]
            zero = zero.sample(frac=drop_zero)
            return joint.drop(index=zero.index)
        
        return joint

<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Sriram\AppData\Local\Temp/ipykernel_6012/1341071288.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if name is 'training':
C:\Users\Sriram\AppData\Local\Temp/ipykernel_6012/1341071288.py:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif name is 'validation':
C:\Users\Sriram\AppData\Local\Temp/ipykernel_6012/1341071288.py:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif name is 'test':  # test


In [5]:
# loading dataset:

# discards the given attributes
celeba = CelebA(drop_features=[
    'Attractive',
    'Pale_Skin',
    'Blurry',
])

# loads all attributes (40)
#celeba = CelebA()

In [6]:
celeba.attributes

,5_o_Clock_Shadow,Arched_Eyebrows,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Brown_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_id
image_id,,,,,,,,,,,,,,,,,,,,,
000001.jpg,0,1,0,0,0,0,0,0,0,1,...,1,1,0,1,0,1,0,0,1,000001.jpg
000002.jpg,0,0,1,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,000002.jpg
000003.jpg,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,000003.jpg
000004.jpg,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,1,000004.jpg
000005.jpg,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,000005.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202595.jpg,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,202595.jpg
202596.jpg,0,0,0,0,1,1,0,0,1,0,...,1,1,0,0,0,0,0,0,1,202596.jpg
202597.jpg,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,202597.jpg


In [7]:
celeba.num_features

37

# 2. Model Architecture
---
* MobileNetV2: as the base architecture pre-trained on 'imagenet'
* Summary: layers, num of parameters.

In [8]:
img_size = 224

IMG_W = img_size 
IMG_H = img_size
IMG_SHAPE = (IMG_H, IMG_W, 3)
TARGET_SIZE = (IMG_H, IMG_W)

In [9]:
# base = MobileNetV2(input_shape=IMG_SHAPE,
#                    weights=None,
#                    include_top=False,
#                    pooling='avg')

In [10]:
# base.summary()

In [11]:
# # model top
# x = base.output
# x = Dense(1536, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.3)(x)
# top = Dense(celeba.num_features, activation='sigmoid')(x)

In [12]:
# model = Model(inputs=base.input, outputs=top)
# model.summary()

# 3. Training
---
* Data generators (for training and validation)
* Image Augmentation: zoom, rotation, shear, shift.
* Optimizer
* Checkpointing
* Fitting

In [13]:
batch_size = 80
num_epochs = 12

In [14]:
# # ------------------------------------------------------------------------------
# # -- Preparing Data Generators for training and validation set
# # ------------------------------------------------------------------------------

# # data augmentation only for the training istances:
# train_datagen = ImageDataGenerator(rotation_range=20, 
#                                    rescale=1./255, 
#                                    width_shift_range=0.2, 
#                                    height_shift_range=0.2, 
#                                    shear_range=0.2, 
#                                    zoom_range=0.2, 
#                                    horizontal_flip=True, 
#                                    fill_mode='nearest')

# valid_datagen = ImageDataGenerator(rescale=1./255)

# # get training and validation set:
# train_split = celeba.split('training'  , drop_zero=False)
# valid_split = celeba.split('validation', drop_zero=False)

# # data generators:
# train_generator = train_datagen.flow_from_dataframe(
#     dataframe=train_split,
#     directory=celeba.images_folder,
#     x_col='image_id',
#     y_col=celeba.features_name,
#     target_size=TARGET_SIZE,
#     batch_size=batch_size,
#     class_mode='other'
# )

# valid_generator = valid_datagen.flow_from_dataframe(
#     dataframe=valid_split,
#     directory=celeba.images_folder,
#     x_col='image_id',
#     y_col=celeba.features_name,
#     target_size=TARGET_SIZE,
#     batch_size=batch_size,
#     class_mode='other'
# )

In [15]:
# # ------------------------------------------------------------------------------
# # -- Compile model
# # ------------------------------------------------------------------------------

# model.compile(loss='cosine_proximity',
#               optimizer='adadelta',  # adadelta, adam, nadam
#               metrics=['binary_accuracy'])

In [16]:
# # ------------------------------------------------------------------------------
# # -- Checkpointing: at each epoch, the best model so far is saved
# # ------------------------------------------------------------------------------

# # model_path = f"{save_path}/UL19/weights-FC37-MobileNetV2-" + "{val_binary_accuracy:.2f}.hdf5"
# model_path = f"weights/weights-FC40-MobileNetV2" + "{val_binary_accuracy:.2f}.hdf5"

# checkpoint = ModelCheckpoint(
#     model_path,
#     monitor='val_binary_accuracy',
#     save_best_only=True,
#     mode='max',
#     verbose=1
# )

In [17]:
# # ------------------------------------------------------------------------------
# # -- Fitting
# # ----------------------------------------------------------------------------
# history = model.fit_generator(
#     train_generator,
#     epochs=num_epochs,
#     steps_per_epoch=len(train_generator),
#     validation_data=valid_generator,
#     validation_steps=len(valid_generator),
#     max_queue_size=1,
#     shuffle=True,
#     callbacks=[checkpoint],
#     verbose=1
# )

# 4. Testing
---
* Test Data Generator
* Evaluate performance on test-set

In [18]:
os.getcwd()

'D:\\Softwares\\Deep_Learning\\Tasks\\Attribute Prediction\\face-clustering'

In [19]:
# optionally load a pre-trained model:

#model = load_model(f"weights/weights-FC37-MobileNetV2-0.92.hdf5", custom_objects={'cosine_proximity': cosine_proximity})
model = load_model(f"weights/weights-FC37-MobileNetV2-0.92.hdf5", compile=False)

In [20]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [21]:
#model.get_weights()

In [22]:
for layer in model.layers:
    layer.trainable = False

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [24]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = celeba.split('test', drop_zero=False)
test_set

,5_o_Clock_Shadow,Arched_Eyebrows,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Brown_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_id
image_id,,,,,,,,,,,,,,,,,,,,,
182638.jpg,0,1,1,0,0,0,0,0,0,0,...,1,0,1,1,1,0,0,0,0,182638.jpg
182639.jpg,0,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,1,0,0,1,182639.jpg
182640.jpg,0,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,0,1,182640.jpg
182641.jpg,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,182641.jpg
182642.jpg,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,182642.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202595.jpg,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,202595.jpg
202596.jpg,0,0,0,0,1,1,0,0,1,0,...,1,1,0,0,0,0,0,0,1,202596.jpg
202597.jpg,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,202597.jpg


In [25]:
test_set = test_set.iloc[:2000]
test_set

,5_o_Clock_Shadow,Arched_Eyebrows,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Brown_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_id
image_id,,,,,,,,,,,,,,,,,,,,,
182638.jpg,0,1,1,0,0,0,0,0,0,0,...,1,0,1,1,1,0,0,0,0,182638.jpg
182639.jpg,0,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,1,0,0,1,182639.jpg
182640.jpg,0,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,0,1,182640.jpg
182641.jpg,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,182641.jpg
182642.jpg,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,182642.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184633.jpg,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,184633.jpg
184634.jpg,0,1,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,0,0,1,184634.jpg
184635.jpg,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,184635.jpg


In [26]:
# ------------------------------------------------------------------------------
# -- Data Generator for Test-Set
# ------------------------------------------------------------------------------



test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_set,
    directory=celeba.images_folder,
    x_col='image_id',
    y_col=celeba.features_name,
    target_size=TARGET_SIZE,
    batch_size=20,
    class_mode='other'
)

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\logging\__init__.py", line 663, in format
    record.message = record.getMessage()
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\logging\__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sriram\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.

Found 2000 validated image filenames.


In [27]:
# # ------------------------------------------------------------------------------
# # -- Evaluate Accuracy
# # ------------------------------------------------------------------------------

# score = model.evaluate_generator(
#     test_generator,
#     steps=len(test_generator),
#     max_queue_size=1,
#     verbose=1
# )

# print("Test score:", score[0])
# print("Test accuracy:", score[1])

In [28]:
#pred = model.predict(x=img, batch_size=1, verbose=1)
predictions = model.predict(x=test_generator, steps=len(test_generator), verbose=1)

100/100 [==============================] - 83s 808ms/step


In [29]:
predictions.shape

(2000, 37)

In [30]:
for i in predictions:
    print(i)

[0.00740114 0.5970065  0.25857383 0.00508252 0.00861838 0.17253667
 0.19885716 0.29803568 0.02708393 0.21710789 0.14584133 0.01832896
 0.00662282 0.00220737 0.00942823 0.00615335 0.9544609  0.791182
 0.00205502 0.9590423  0.00890374 0.04036137 0.98800606 0.7206416
 0.12901634 0.01850832 0.00411662 0.00353217 0.9735793  0.70895505
 0.10424006 0.7211118  0.00286177 0.97474533 0.12645155 0.00571781
 0.99206257]
[0.7500894  0.02990216 0.87776566 0.02070624 0.00363061 0.03981459
 0.6405712  0.02583674 0.00741842 0.13120484 0.0162178  0.01457709
 0.03025097 0.00250015 0.08814782 0.02064961 0.00739968 0.08734348
 0.99946    0.02286896 0.03532082 0.00580347 0.17009598 0.24860695
 0.36096728 0.86436605 0.00369015 0.03819069 0.15184084 0.11803541
 0.03045708 0.00284085 0.00287825 0.00959864 0.01122442 0.06684902
 0.13216963]
[5.4059625e-03 8.4722370e-01 9.0052158e-02 5.8116317e-03 1.4178187e-02
 3.3910495e-01 1.0876545e-01 1.8643737e-02 9.3885434e-01 5.5933595e-03
 8.3111525e-03 7.4067414e-03 1.

[1.10909045e-02 3.03080678e-03 5.51511347e-02 1.27823353e-02
 3.25003266e-03 7.78430700e-03 1.31651163e-02 4.63388562e-02
 2.37619579e-02 8.23208988e-02 3.43805552e-03 3.63624096e-03
 1.00472569e-03 4.31269407e-04 4.65527177e-03 4.87035513e-03
 6.73341751e-03 8.71160626e-02 5.26966393e-01 1.60287231e-01
 1.24427676e-03 1.01980567e-02 9.91198897e-01 3.06640327e-01
 8.34372640e-02 6.59204125e-02 2.12103128e-03 2.44170427e-03
 4.32893634e-03 6.79080486e-02 1.59050822e-02 2.35546827e-02
 1.01934373e-02 5.17541170e-03 7.04312325e-03 3.61502171e-03
 9.70933855e-01]
[2.8661191e-03 8.7141335e-01 1.4015496e-02 2.6413798e-03 1.8515587e-03
 6.9100559e-01 2.6121557e-02 1.4632225e-02 9.9027157e-04 6.9066840e-01
 1.7019868e-02 2.7573705e-03 1.3371706e-03 3.5912693e-03 4.7377944e-03
 1.7118752e-03 9.9529797e-01 2.5662345e-01 1.8899318e-05 1.0003388e-02
 4.2135417e-03 3.2279789e-02 9.9776018e-01 1.3023558e-01 1.6594318e-01
 1.3143986e-02 1.4607400e-02 3.7825704e-03 1.4486194e-02 6.7081362e-02
 8.55327

 0.9916996 ]
[0.53659254 0.03991336 0.22672197 0.00554654 0.00555959 0.04086572
 0.13056034 0.01345769 0.00914276 0.05873165 0.0353353  0.01534832
 0.00802514 0.00119993 0.74006915 0.00290564 0.01488498 0.0092341
 0.99330604 0.0054321  0.27296558 0.02143005 0.00740129 0.14791158
 0.7852817  0.01301068 0.01172525 0.80070084 0.02529386 0.07164055
 0.02207991 0.00420186 0.99319077 0.00887355 0.01812205 0.03159118
 0.8967562 ]
[0.0431132  0.00832984 0.02977526 0.01240283 0.02356201 0.04629573
 0.07163128 0.653568   0.00593531 0.02328268 0.00795138 0.00857547
 0.00281408 0.9847069  0.00651383 0.00299358 0.0038307  0.02355856
 0.985233   0.00900951 0.00606993 0.03633204 0.9626611  0.10876283
 0.13284704 0.01934567 0.00227267 0.00229758 0.01238549 0.19397107
 0.03380588 0.05933428 0.00689441 0.00331894 0.02964225 0.00819945
 0.95048803]
[0.00377697 0.04497978 0.00781855 0.00377044 0.9462743  0.5078326
 0.03509551 0.03800938 0.51185644 0.01102117 0.00513214 0.00381431
 0.00234085 0.03396466 0.

 0.15292713]
[1.34496987e-02 1.04194790e-01 8.14355016e-01 3.04578841e-02
 1.18566453e-02 7.70313263e-01 9.81430769e-01 7.89340138e-01
 5.79684973e-04 1.07704103e-02 1.43864602e-01 7.65203655e-01
 2.12829024e-01 9.16242599e-04 9.73726630e-01 3.08215618e-03
 1.33744180e-02 9.39760923e-01 9.97563004e-01 4.22534943e-02
 5.40229678e-01 1.89028084e-02 5.81389666e-03 7.91589975e-01
 1.09678805e-02 6.35683894e-01 6.76387548e-03 8.32418978e-01
 9.13988471e-01 6.60749376e-02 3.38418186e-02 4.37590182e-02
 2.97024846e-03 1.40379667e-02 2.03267336e-02 3.20186913e-01
 6.92467451e-01]
[0.00793773 0.16995257 0.44572625 0.00292051 0.04283494 0.18924946
 0.17103958 0.31571567 0.0091005  0.10920444 0.18723845 0.00902107
 0.00532162 0.00181678 0.01090187 0.0100233  0.55940753 0.6523372
 0.0034889  0.9930934  0.0054692  0.36211386 0.98591185 0.28445858
 0.14864653 0.00539747 0.00561139 0.00602055 0.9641812  0.08351839
 0.32684642 0.9571036  0.0106726  0.6935405  0.560325   0.0013113
 0.9599196 ]
[0.65065

[8.3866417e-03 4.1923499e-01 2.1198267e-01 2.6569963e-03 1.8973589e-02
 3.5229009e-01 2.4700195e-02 9.4268888e-02 3.2821298e-03 2.0892778e-01
 3.0122846e-02 2.4925172e-03 1.0590851e-03 1.8116236e-03 6.9884956e-03
 3.3501983e-03 8.0845296e-01 2.7771294e-02 9.5996261e-04 7.2289407e-03
 5.0500631e-03 2.6369154e-02 9.9491811e-01 1.8687525e-01 3.3009061e-01
 4.4784814e-02 2.2953749e-03 7.2383583e-03 8.1571341e-03 2.2085279e-02
 6.4766091e-01 3.0450696e-01 3.2487214e-03 9.0242314e-01 1.4944419e-01
 2.3540854e-03 9.9676669e-01]
[6.5064430e-04 2.5439918e-02 1.8963718e-01 3.9619207e-03 2.6793510e-02
 1.0133377e-01 8.7763160e-02 1.6260713e-02 6.3537896e-02 7.1695089e-02
 2.7333796e-03 3.0063480e-02 4.9793482e-02 1.2143642e-02 3.1427741e-03
 2.3371968e-01 9.6538961e-03 4.4256371e-01 3.3148825e-03 1.6132683e-02
 2.6309788e-03 2.9880577e-01 9.9888921e-01 1.2402779e-01 1.0669035e-01
 1.1556995e-01 4.9175620e-03 1.6364753e-03 6.2260306e-01 1.4676243e-02
 7.9474354e-01 3.7368953e-02 3.4956038e-03 1.04

...

[0.00553784 0.4607931  0.2091417  0.00611356 0.00365117 0.41560617
 0.04685462 0.02009761 0.91713023 0.00451833 0.00604558 0.00228277
 0.00266406 0.00835627 0.00565344 0.00994951 0.54385245 0.09187001
 0.0018962  0.7871163  0.00385264 0.02891564 0.9982592  0.05104399
 0.5061364  0.06245318 0.00546905 0.00317538 0.01353434 0.1838358
 0.23016524 0.04871953 0.00280842 0.782802   0.15356249 0.00305721
 0.96891874]
[7.2108805e-03 1.4537483e-02 8.3978713e-02 1.1979640e-03 5.7204664e-03
 1.1802840e-01 3.7591487e-02 1.5855911e-01 7.4466169e-03 4.7857672e-02
 5.3237230e-02 2.3819208e-03 9.6634030e-04 9.6642971e-04 2.3208559e-03
 1.0828674e-03 4.3068230e-03 1.3192207e-02 4.2658448e-03 4.3833435e-02
 4.5839548e-03 4.2913795e-02 9.9670422e-01 1.4567879e-01 9.7152770e-02
 6.0846984e-02 2.2699237e-03 3.7252307e-03 9.9673867e-03 1.8998915e-01
 1.0460526e-02 2.5167376e-02 1.2886345e-02 3.3363521e-02 5.2763134e-02
 2.2332370e-03 9.9926615e-01]
[2.0315945e-02 3.4228316e-01 3.4431201e-01 1.5935898e-03 6.

 0.9721366 ]
[0.8916234  0.00362638 0.25847554 0.00287262 0.00924346 0.05649811
 0.0648939  0.03864038 0.00377038 0.37681767 0.03057835 0.01209459
 0.00835329 0.0029926  0.00482112 0.00150043 0.00287813 0.01009527
 0.9993161  0.00257596 0.01038453 0.48563468 0.29422712 0.10102203
 0.20664367 0.03174311 0.00476921 0.05880573 0.0150997  0.34160852
 0.03764689 0.0025858  0.00429076 0.00224617 0.00680628 0.06207514
 0.9819284 ]
[1.6334206e-02 8.2489371e-02 3.9177552e-01 5.1888824e-03 9.9481320e-01
 3.2958144e-01 5.3629404e-01 3.6561787e-03 3.7036747e-02 7.0884430e-01
 1.9296676e-02 3.0691445e-02 1.6217017e-01 3.6420226e-03 9.9297464e-03
 1.3980597e-02 8.3239329e-01 9.7858036e-01 4.0732920e-03 9.9715114e-01
 8.2997382e-03 4.0036693e-01 9.8347270e-01 7.1573853e-02 2.8375560e-01
 6.4418614e-03 4.3805146e-01 2.0487130e-02 9.9824345e-01 8.0313385e-03
 8.8387263e-01 9.6259391e-01 1.8980801e-03 9.8600030e-01 7.6885217e-01
 9.9557638e-04 6.5271258e-03]
[7.5632930e-03 2.4248391e-02 2.0341682e-01 6.

[0.0123851  0.05085817 0.27757692 0.00337967 0.01167163 0.12720707
 0.12220192 0.8197362  0.00320673 0.03581047 0.01524809 0.00957477
 0.00302428 0.00663313 0.00341985 0.00287658 0.01543546 0.33578444
 0.02190456 0.8334596  0.00455937 0.06192011 0.99735445 0.10667557
 0.21444085 0.20158085 0.00409469 0.00223446 0.0159412  0.08206862
 0.11465317 0.45544443 0.00259027 0.03240785 0.05868569 0.01093879
 0.9390635 ]
[5.3795874e-03 8.2728755e-01 9.2971027e-03 3.7774146e-03 1.5115172e-02
 2.5605929e-01 2.7959853e-02 3.2840797e-01 1.1166066e-02 1.1432856e-01
 2.7663261e-02 4.8501790e-03 5.0505698e-03 1.6739070e-03 1.0390222e-02
 4.9670935e-03 9.9878252e-01 9.8684645e-01 4.7695637e-04 9.7777569e-01
 3.7062466e-03 6.8452835e-01 9.9081099e-01 4.7167557e-01 5.1558077e-01
 1.4363885e-02 2.0420995e-01 5.2450597e-03 9.9611348e-01 8.9476377e-02
 6.6620392e-01 9.5182621e-01 6.0528517e-04 9.9693537e-01 2.4251264e-01
 1.7266572e-03 9.4736671e-01]
[0.35065293 0.00892308 0.03347701 0.01732284 0.00352988 0.

[6.8269670e-03 4.0793544e-01 2.6029229e-02 1.1395633e-02 1.8177900e-01
 3.9510173e-01 2.0874023e-02 2.7260095e-02 9.9746132e-01 1.3757944e-03
 1.2490451e-02 2.9551983e-03 2.5748909e-03 1.5409291e-03 1.1012018e-02
 4.3383837e-03 9.9945128e-01 9.9385762e-01 1.5148520e-04 9.9135709e-01
 5.0493777e-03 2.9075164e-01 9.9475819e-01 6.6050339e-01 6.2856573e-01
 4.6934187e-03 1.8018976e-01 4.5154989e-03 9.9949908e-01 8.8866490e-01
 1.2792069e-01 3.1060505e-01 1.9454956e-03 9.9864304e-01 2.2451961e-01
 3.5851300e-03 9.9435139e-01]
[0.24603307 0.03386489 0.09130815 0.00487131 0.00786644 0.893087
 0.7826221  0.9878982  0.00390381 0.005968   0.8624782  0.09324124
 0.00964269 0.00667998 0.04378054 0.00108221 0.02341118 0.37104917
 0.9990382  0.52417386 0.05724975 0.02051136 0.4577412  0.74408406
 0.02695623 0.04155308 0.00343454 0.00402245 0.13530952 0.15499687
 0.01451805 0.13795781 0.00285026 0.00923041 0.01459709 0.03057525
 0.9939851 ]
[0.02289176 0.01292098 0.9075459  0.08146405 0.01083952 0.02

 0.94042015]
[0.00539857 0.24138051 0.03399447 0.00285071 0.00308034 0.18417704
 0.02285704 0.05238661 0.00630367 0.10916823 0.01233599 0.00420004
 0.00141034 0.00291047 0.00284287 0.00204015 0.41735375 0.0091832
 0.00126678 0.00720271 0.00471705 0.01967639 0.99650645 0.21648899
 0.5255592  0.4602836  0.00533989 0.00507963 0.00211206 0.19142568
 0.01014453 0.5962337  0.00234231 0.42042884 0.06575036 0.00403717
 0.9977621 ]
[0.0060471  0.02525085 0.06556374 0.00187007 0.00765529 0.2097587
 0.01643851 0.02373868 0.11929226 0.11094519 0.01030767 0.00351277
 0.00331366 0.02772555 0.00176257 0.00583839 0.04133099 0.03786436
 0.00992224 0.30005962 0.00586307 0.03006598 0.9945452  0.0260489
 0.6260348  0.03879258 0.00236174 0.00199875 0.05812433 0.3103848
 0.05629605 0.31923068 0.00225991 0.0808368  0.02856559 0.00643492
 0.97441936]
[1.03995025e-01 5.13541698e-03 1.09933078e-01 4.29618359e-03
 7.61359930e-03 9.11732614e-02 7.49212801e-02 6.15571320e-01
 7.50294328e-03 1.82713568e-02 2.009530

In [31]:
# convert labels to 0, 1 integers.
preds = np.round(predictions).astype('int')
for i in preds:
    print(i)

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 1]
[1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1]
[1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
[0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1]
[1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0

[0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1]
[0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1]
[0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 1]
[1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 1]
[0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1]
[1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1

[0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1]
[0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1]
[0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1]
[0 0 0 0 1 0

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1]
[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 1 0 1 0

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
[1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1]
[0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0]
[1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1]
[0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 1 0 0 1]
[0 0 0 0 0 0

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1]
[0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1]
[1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 0

In [32]:
dir(test_generator)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_params',
 '_filepaths',
 '_filter_classes',
 '_filter_valid_filepaths',
 '_flow_index',
 '_get_batches_of_transformed_samples',
 '_keras_api_names',
 '_keras_api_names_v1',
 '_sample_weight',
 '_set_index_array',
 '_targets',
 'allowed_class_modes',
 'batch_index',
 'batch_size',
 'class_mode',
 'color_mode',
 'data_format',
 'directory',
 'dtype',
 'filenames',
 'filepaths',
 'get_classes',
 'image_data_generator',
 'image_shape',
 'index_array',
 'index_generator',
 'interpolation',
 'labels',
 'lock',
 'n',
 'next',
 'on_epoch_end',
 'reset',
 'sample_weight

In [33]:
celeba.features_name

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mouth_Slightly_Open',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young']

In [34]:
test_generator.labels

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [35]:
test_generator.labels.shape

(2000, 37)

In [36]:
# !pip install seaborn

In [37]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import itertools
import shutil
import random
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [38]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_true=test_generator.labels, y_pred=preds)

array([[[1644,  156],
        [ 181,   19]],

       [[1178,  216],
        [ 520,   86]],

       [[1240,  368],
        [ 309,   83]],

       [[1899,   53],
        [  48,    0]],

       [[1443,  245],
        [ 255,   57]],

       [[1173,  131],
        [ 624,   72]],

       [[1270,  308],
        [ 339,   83]],

       [[1038,  358],
        [ 440,  164]],

       [[1570,  186],
        [ 223,   21]],

       [[1379,  247],
        [ 320,   54]],

       [[1629,  116],
        [ 240,   15]],

       [[1835,   48],
        [ 112,    5]],

       [[1846,   54],
        [  98,    2]],

       [[1732,  128],
        [ 126,   14]],

       [[1822,   87],
        [  88,    3]],

       [[1872,   61],
        [  64,    3]],

       [[ 708,  468],
        [ 521,  303]],

       [[ 582,  446],
        [ 546,  426]],

       [[ 720,  509],
        [ 481,  290]],

       [[ 525,  476],
        [ 493,  506]],

       [[1866,   54],
        [  80,    0]],

       [[1635,   69],
        [ 28

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_true=test_generator.labels, y_pred=preds, target_names=celeba.features_name))

                     precision    recall  f1-score   support

   5_o_Clock_Shadow       0.11      0.10      0.10       200
    Arched_Eyebrows       0.28      0.14      0.19       606
    Bags_Under_Eyes       0.18      0.21      0.20       392
               Bald       0.00      0.00      0.00        48
              Bangs       0.19      0.18      0.19       312
           Big_Lips       0.35      0.10      0.16       696
           Big_Nose       0.21      0.20      0.20       422
         Black_Hair       0.31      0.27      0.29       604
         Blond_Hair       0.10      0.09      0.09       244
         Brown_Hair       0.18      0.14      0.16       374
     Bushy_Eyebrows       0.11      0.06      0.08       255
             Chubby       0.09      0.04      0.06       117
        Double_Chin       0.04      0.02      0.03       100
         Eyeglasses       0.10      0.10      0.10       140
             Goatee       0.03      0.03      0.03        91
          Gray_Hair    

In [40]:
classification_report_file = classification_report(y_true=test_generator.labels, y_pred=preds, target_names=celeba.features_name, output_dict=True)
print(classification_report_file)

{'5_o_Clock_Shadow': {'precision': 0.10857142857142857, 'recall': 0.095, 'f1-score': 0.10133333333333333, 'support': 200}, 'Arched_Eyebrows': {'precision': 0.2847682119205298, 'recall': 0.1419141914191419, 'f1-score': 0.1894273127753304, 'support': 606}, 'Bags_Under_Eyes': {'precision': 0.18403547671840353, 'recall': 0.21173469387755103, 'f1-score': 0.19691577698695137, 'support': 392}, 'Bald': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 48}, 'Bangs': {'precision': 0.18874172185430463, 'recall': 0.18269230769230768, 'f1-score': 0.18566775244299674, 'support': 312}, 'Big_Lips': {'precision': 0.35467980295566504, 'recall': 0.10344827586206896, 'f1-score': 0.16017797552836485, 'support': 696}, 'Big_Nose': {'precision': 0.21227621483375958, 'recall': 0.1966824644549763, 'f1-score': 0.20418204182041821, 'support': 422}, 'Black_Hair': {'precision': 0.31417624521072796, 'recall': 0.271523178807947, 'f1-score': 0.2912966252220249, 'support': 604}, 'Blond_Hair': {'precision': 

In [41]:
from sklearn import metrics
print("None ", metrics.precision_score(y_true=test_generator.labels, y_pred=preds, average=None))

None  [0.10857143 0.28476821 0.18403548 0.         0.18874172 0.3546798
 0.21227621 0.31417625 0.10144928 0.17940199 0.11450382 0.09433962
 0.03571429 0.09859155 0.03333333 0.046875   0.39299611 0.48853211
 0.36295369 0.51527495 0.         0.14814815 0.8524971  0.27759197
 0.28988764 0.0989011  0.04597701 0.04545455 0.51253761 0.24939467
 0.38321995 0.2283237  0.05063291 0.48701299 0.17045455 0.05555556
 0.76359194]


In [42]:
precision_score = metrics.precision_score(y_true=test_generator.labels, y_pred=preds, average=None)
type(precision_score)

numpy.ndarray

In [43]:
#1) np.savetxt('classification_report.txt',classification_report,delimiter=',')
#2) with open('classification_report.txt') as f:
#     f.write(str(classification_report))
#     f.write('\n\nPrecision_score\n')
#     f.write(str(precision_score))
# report = np.loadtxt('classification_report.txt',delimiter=',')
# print(report)
#3)
classification_report_dffile = pd.DataFrame(classification_report_file).transpose()
classification_report_dffile.to_csv('classification_report.csv', index= True)

In [44]:
import csv
with open('classification_report.txt', "w") as f_o:
    with open('classification_report.csv', "r") as f_i:
        [ f_o.write(" ".join(row)+'\n') for row in csv.reader(f_i)]
    f_o.close()